In [1]:
#if needed, go for it
#!pip install folium
#!pip install geopy
#!conda install -c conda-forge xarray dask netCDF4 bottleneck

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import os 
from datetime import datetime as dt
from datetime import timedelta
from geopy.distance import geodesic
import folium
from folium import plugins
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Drifter Data analysis between a year gap in the same area
This analysis is done for the 12 of october 2021 and 2022, for other days the proccess is roughly the same

#### Here we define some functions

In [3]:
def ordinal_to_dt(time):
    day = time%1 * 24
    hour = day%1 * 60
    seconds = hour%1 *60
    
    return dt.strptime(f"{int(day)}:{int(hour)}:{int(seconds)}", '%H:%M:%S') 

ordinal_to_dt = np.vectorize(ordinal_to_dt)

In [4]:
def compute_velocities(df):
    #We compute the geodesic distance using geopy function, for all the values of this drifter
    #The new column we are creating its going to be filled with the array defined in the for loop
    df['distance'] = [ 
        #compute the distance
        geodesic(
            [df.loc[i-1,'Latitude'],df.loc[i-1,'Longitude']], #pos i-1
            [df.loc[i,'Latitude'],df.loc[i,'Longitude']] #pos i
        ).m
        if i>0 #not for the first element to avoid errors
        else np.finfo(float).eps #instead we fill it with an infinitesimal value to avoid dividing by 0
        for i in range(len(df)) #for i position such as i is the number of elements
    ]
    #compute the time delta
    df['time_delta'] = [ 
        #by resting time(i-1) and time(i) and gettings the seconds of it
        np.abs((df.loc[i-1,'DeviceDateTime']-df.loc[i,'DeviceDateTime']).total_seconds())
        if i>0
        else 1
        for i in range(len(df))
    ]
    #The velocity will be de  division of this values
    df['velocity'] = df['distance']/df['time_delta']

    # apply normalization techniques for velocity visualization
    df['vel_normalized'] = MinMaxScaler().fit_transform(np.array(df['velocity']).reshape(-1,1))

    return df

### Data 2021 
#### Now we read the data from 2021 and compute the velocities
We will get a dataframe per drifter, also we have deployment information in the logsheet

In [5]:
#Read the log sheet and clean the id to get only 3 digits identifier
logsheet_2021 = pd.read_csv('./Drifter-2021/logsheet_MIR_12102021.csv',delimiter=";")
logsheet_2021['Flotteur_ID'] = logsheet_2021['Flotteur_ID'].apply(lambda x: str(x)[-3:])

#list the files for the 12-10-2021
drifters_files_2021 = [i  for i in os.listdir('./Drifter-2021') if '12102021' in i and 'nc' in i]
drifters_2021 = {}

for file in drifters_files_2021:
    
    #get id (only last 3 digits)
    drifter_id = f'{file.split(".")[-2].split("_")[-1][-3:]}'
    #open nc
    xr_df = xr.open_dataset(f'./Drifter-2021/{file}')
    #now lets convert it to pandas
    df = xr_df.to_dataframe()
    
    #new column to identify our friend
    df['DeviceName'] = drifter_id
    #convert ordinal time to datetime
    df['TIME'] = ordinal_to_dt(df['TIME'].to_numpy())
    #rename so all drifter from all year follow the same naming
    df = df.rename(
        columns={
            "TIME": "DeviceDateTime",
            "LATITUDE" : "Latitude",
            "LONGITUDE" : "Longitude"
        }
    )
    #like in a sunday morning: forget what needs to be forgotten
    df = df.drop(columns=[ 'DROGUE_DEPTH','BUOY_ID','STATIONNAME'])
    #Sort the DataFrame by the time and reset the index
    df = df.sort_values(by='DeviceDateTime').reset_index()
    #Compute velocities
    df = compute_velocities(df)
    #Save it!
    drifters_2021[drifter_id] = df


### Data 2022
#### Now we go for the data of 2022
We read the logsheet and then compute the velocities in the same way

In [6]:
logSheet = pd.read_csv(f'logSheet.csv',encoding='utf8') 

#Remove the id number extra digits, to get only the last 3 digits od the id, it is what we are using
logSheet['Id'] = logSheet['Id'].apply(lambda x: x[-3:])
#We want to convert the time as string to a datetime timestamp for both recover and deploy
logSheet['Deplyoment'] = [
    dt.strptime(f'{logSheet.iloc[i]["Deplyoment"]}:00', '%H:%M:%S') - timedelta(hours=2, minutes=0)
    for i in range(len(logSheet))
]

logSheet['Recovery'] = [
    dt.strptime(f'{logSheet.iloc[i]["Recovery"]}:00', '%H:%M:%S') - timedelta(hours=2, minutes=0)
    for i in range(len(logSheet))
]
#Now we can take a look at it
logSheet.head()

,Station,Id,Type,Deplyoment,Lon,Lat,Recovery,Lon.1,Lat.1
0,1,273,Yellow 1m,1900-01-01 08:28:00,5º 59.928',43º 4.8',1900-01-01 12:05:00,5º 59.994,43º 5.342'
1,1,666,White surface,1900-01-01 08:28:00,5º 59.928',43º 4.8',1900-01-01 12:13:00,6º 0.194',43º 5.617'
2,1,439,White 60cm,1900-01-01 08:28:00,5º 59.928',43º 4.8',1900-01-01 12:10:00,5º 59.399',43º 5.403'
3,2,274,Yellow 1m,1900-01-01 08:42:00,5º 59.072',43º 4.695',1900-01-01 11:39:00,5º 59.423',43º 4.929'
4,2,436,White surface,1900-01-01 08:42:00,5º 59.072',43º 4.695',1900-01-01 11:38:00,5º 59.282',43º 4.958


#### But first we need to merge both types of datasets, since we have different format for 2022 data 

In [7]:
#First we read the white drifter data from the single csv containing all of them
df_all = pd.read_csv(f'./MIR_DAY1/MIR_DAY1_DRIFTERS/drifter-10_11_22-05_49.csv',encoding='utf8')
#We remove the data we are not goin to use
df_all = df_all.drop(['BatteryStatus','CommId'], axis=1)
#We remove erroneous data
df_all = df_all.dropna()

#We iterate over the data folder and we only get the yellow drifters files (txt files)
for file in [i for i in os.listdir('./MIR_DAY1/MIR_DAY1_DRIFTERS') if i[-3:]=='txt']:
    #We get the id
    drifter_id = f'{file.split(".")[-2][-3:]}'#split filename by ".", get the second last, and get 3 last characters
    
    #read the csv, yes its a csv even if its termination its txt
    df = pd.read_csv(f'./MIR_DAY1/MIR_DAY1_DRIFTERS/{file}',encoding='utf8')
    #We rename the time column like the white data is
    df = df.rename(columns={"Reception time (UTC)": "DeviceDateTime"})
    #create a new colunm with the id
    df['DeviceName'] = drifter_id
    #We remove the data we are not goin to use
    df = df.drop(['Position time (UTC)', 'Course (°)','Speed (m/s)','Status','Battery (V)','Temperature (°C)'], axis=1)
    #order the columns like the white ones
    df = df.loc[:, ["DeviceName","DeviceDateTime","Latitude","Longitude"]]
    #Append what we have to the big dataframe
    df_all = df_all.append(df)
    

#### Now we can apply the same processing for all of them
Please always execute the cell above before this step

In [8]:
#we create some dicts to store the clean data sorted by its type
yellow_drifters_df = {}
white_drifters_surface_df = {}
white_drifters_60_df = {}


#Cast date string to python datetime only 'caring' about the hours
df_all['DeviceDateTime'] = df_all['DeviceDateTime'].apply(lambda x: dt.strptime(x.split()[-1], '%H:%M:%S')) 
#Remove the id number extra digits, to get only the last 3 digits od the id, it is what we are using
df_all['DeviceName'] = df_all['DeviceName'].apply(lambda x: x[-3:])

#Sort the DataFrame by the time and reset the index
df_all=df_all.sort_values(by='DeviceDateTime').reset_index()


#We iterate over the logSheet entries
for index, row in logSheet.iterrows():
    
    #Filter the df to get only the signal after deployment and before recover
    df = df_all.loc[
        (df_all['DeviceName'] ==  row['Id']) # Id that we want
        & #and
        (df_all['DeviceDateTime'] > row['Deplyoment']) # DeviceDateTime is greater than deploy
        & #and
        (df_all['DeviceDateTime'] < row['Recovery']) # DeviceDateTime is smaller than recover
    ]
    #sort and reset index again! 
    df=df.sort_values(by='DeviceDateTime').reset_index()

    #We make sure that there is some data
    if len(df)>0:
        
        df = compute_velocities(df)

        #Depending of its type we put it in a place or another
        if '60' in row['Type']:
            white_drifters_60_df[row['Id']]=df
        elif 'surface' in row['Type']:
            white_drifters_surface_df[row['Id']]=df
        elif '1m' in row['Type']:
            yellow_drifters_df[row['Id']]=df

### Data visualization
This need to be improved  for sure

#### 2022

In [9]:
#We create a map object using folium
m = folium.Map(location=[43.05228, 5.94], zoom_start=12, tiles="Stamen Terrain")
#We add a raster layer using a wms service from emodnet thats provide bathymetry maps
folium.raster_layers.WmsTileLayer(url= 'https://ows.emodnet-bathymetry.eu/wms',
                                  layers = 'mean_rainbowcolour',
                                  opacity = 0.3,
                                  transparent = True, 
                                  control = True,
                                  fmt="image/png",
                                  name = 'bathymetry',
                                  overlay = True,
                                  show = True,
                                  ).add_to(m)
#add this last layer to the control
folium.LayerControl().add_to(m)

#for every tyoe of drifter we plot the trayectories
for key,item in white_drifters_60_df.items():
    
    #get some variables
    lats = item['Latitude'].tolist()
    lons = item['Longitude'].tolist()
    vels = item['velocity'].tolist()
    vels_normalized = item['vel_normalized'].tolist()

    #this next lines are to make the path animated, I think its a bit ugly
    #latslons = zip(lats,lons)
    #plugins.AntPath(latslons,color='white').add_to(m)
    
    #for every point
    for i in range(len(item['Latitude'].tolist())):
        
        #plot a marker whit a poppup and its intensity proportional to its velocity
        folium.CircleMarker(
            [lats[i],lons[i] ],
            opacity = vels_normalized[i],
            radius=2,color='white',
            fill_color='black',
            popup=folium.Popup(f"z=0.6m: {round(vels[i],3)}m/s")
        ).add_to(m)


for key,item in white_drifters_surface_df.items():
    
    
    lats = item['Latitude'].tolist()
    lons = item['Longitude'].tolist()
    vels = item['velocity'].tolist()
    vels_normalized = item['vel_normalized'].tolist()

    
    #latslons = zip(lats,lons)
    #plugins.AntPath(latslons,color='#8da6c2').add_to(m)
    
    for i in range(len(item['Latitude'].tolist())):

        folium.CircleMarker(
            [lats[i],lons[i] ],
            opacity = vels_normalized[i],
            radius=2,
            color='#785ebf',
            fill_color='white',
            popup=folium.Popup(f"z=0m: {round(vels[i],3)}m/s")
        ).add_to(m)


for key,item in yellow_drifters_df.items():
    
    lats = item['Latitude'].tolist()
    lons = item['Longitude'].tolist()
    vels = item['velocity'].tolist()
    vels_normalized = item['vel_normalized'].tolist()
    
    #latslons = zip(lats,lons)
    #plugins.AntPath(latslons,color='yellow').add_to(m)

    for i in range(len(item['Latitude'].tolist())):
       
    
        folium.CircleMarker(
            [lats[i],lons[i]],
            opacity = vels_normalized[i],
            radius=2,
            color='#decb40',
            fill_color='yellow',
            popup=folium.Popup(f"z=1m: {round(vels[i],3)}m/s")
        ).add_to(m)
        
title = 'Drifter trayectories, intesities are proportional to velocities'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(title)   


m.get_root().html.add_child(folium.Element(title_html))

m

#### 2021

In [10]:
#We create a map object using folium
m = folium.Map(location=[43.05228, 5.94], zoom_start=12, tiles="Stamen Terrain")
#We add a raster layer using a wms service from emodnet thats provide bathymetry maps
folium.raster_layers.WmsTileLayer(url= 'https://ows.emodnet-bathymetry.eu/wms',
                                  layers = 'mean_rainbowcolour',
                                  opacity = 0.3,
                                  transparent = True, 
                                  control = True,
                                  fmt="image/png",
                                  name = 'bathymetry',
                                  overlay = True,
                                  show = True,
                                  ).add_to(m)
#add this last layer to the control
folium.LayerControl().add_to(m)

#for every tyoe of drifter we plot the trayectories
for key,item in drifters_2021.items():
    
    #get some variables
    lats = item['Latitude'].tolist()
    lons = item['Longitude'].tolist()
    vels = item['velocity'].tolist()
    vels_normalized = item['vel_normalized'].tolist()

    #this next lines are to make the path animated, I think its a bit ugly
    #latslons = zip(lats,lons)
    #plugins.AntPath(latslons,color='white').add_to(m)
    
    #for every point
    for i in range(len(item['Latitude'].tolist())):
        
        #plot a marker whit a poppup and its intensity proportional to its velocity
        folium.CircleMarker(
            [lats[i],lons[i] ],
            opacity = vels_normalized[i],
            radius=2,color='white',
            fill_color='black',
            popup=folium.Popup(f"z=0.6m: {round(vels[i],3)}m/s")
        ).add_to(m)

        
title = 'Drifter trayectories, intesities are proportional to velocities'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(title)   


m.get_root().html.add_child(folium.Element(title_html))

m